In [42]:
import gurobipy as gb
from gurobipy import GRB, quicksum
import pandas as pd

In [43]:
df = pd.read_csv(r"C:\Users\johns\OneDrive\Desktop\MBAN Semester 3\OMIS 6000 - Models & Applications in Operational Research\Midterm\nurse_shift_costs.csv")

In [44]:
costs_weekday = df['Cost_Weekday'].tolist()
costs_weekend = df['Cost_Weekend'].tolist()
costs_overtime = df['Cost_Overtime'].tolist()
nurse_categories = df['Category'].tolist()

In [45]:
# Create a new model
model = gb.Model("ICU_Nurse_Scheduling")

In [46]:
# Constants
NUM_NURSES = 26
NUM_SHIFTS = 14
HOURS_PER_SHIFT = 12

In [47]:
# Binary Variables
# x[i, j] = 1 if nurse i is assigned to shift j
x = model.addVars(NUM_NURSES, NUM_SHIFTS, vtype=GRB.BINARY, name="x")

In [48]:
# Overtime Variables
# o[i, j] = 1 if shift j for nurse i is overtime
o = model.addVars(NUM_NURSES, NUM_SHIFTS, vtype=GRB.BINARY, name="o")

In [49]:
# Objective Function: Minimize total cost
model.setObjective(
    gb.quicksum(x[i, j] * (costs_weekday[i] if j % 7 <= 4 else costs_weekend[i]) for i in range(NUM_NURSES) for j in range(NUM_SHIFTS))
    + gb.quicksum(o[i, j] * costs_overtime[i] for i in range(NUM_NURSES) for j in range(NUM_SHIFTS)),
    GRB.MINIMIZE)

In [50]:
# Constraints

# Each shift must have at least 6 nurses
for j in range(NUM_SHIFTS):
    model.addConstr(gb.quicksum(x[i, j] for i in range(NUM_NURSES)) >= 6, name=f"min_nurses_shift_{j}")

# Each nurse works between 36 and 60 hours per week
for i in range(NUM_NURSES):
    model.addConstr(HOURS_PER_SHIFT * gb.quicksum(x[i, j] for j in range(NUM_SHIFTS)) >= 36, name=f"min_hours_nurse_{i}")
    model.addConstr(HOURS_PER_SHIFT * gb.quicksum(x[i, j] for j in range(NUM_SHIFTS)) <= 60, name=f"max_hours_nurse_{i}")

# Each shift includes at least one SRN
for j in range(NUM_SHIFTS):
    model.addConstr(gb.quicksum(x[i, j] for i in range(NUM_NURSES) if nurse_categories[i] == "SRN") >= 1, name=f"srn_requirement_shift_{j}")

# No back-to-back shifts
for i in range(NUM_NURSES):
    for j in range(1, NUM_SHIFTS):
        model.addConstr(x[i, j] + x[i, j-1] <= 1, name=f"no_back_to_back_{i}_{j}")

# Overtime Constraints
# Total weekly hours for each nurse
weekly_hours = model.addVars(NUM_NURSES, vtype=GRB.CONTINUOUS, name="weekly_hours")

for i in range(NUM_NURSES):
    model.addConstr(weekly_hours[i] == HOURS_PER_SHIFT * quicksum(x[i, j] for j in range(NUM_SHIFTS)), name=f"total_hours_{i}")

    for j in range(NUM_SHIFTS):
        # If total hours exceed 36, mark the shift as overtime
        model.addConstr(o[i, j] >= (weekly_hours[i] - 36)/HOURS_PER_SHIFT - (NUM_SHIFTS - j), name=f"overtime_{i}_{j}")

In [51]:
# Optimize the model
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 808 rows, 754 columns and 3026 nonzeros
Model fingerprint: 0xac1791aa
Variable types: 26 continuous, 728 integer (728 binary)
Coefficient statistics:
  Matrix range     [8e-02, 1e+01]
  Objective range  [1e+02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
Found heuristic solution: objective 35854.000000
Presolve removed 390 rows and 338 columns
Presolve time: 0.00s
Presolved: 418 rows, 416 columns, 1622 nonzeros
Variable types: 0 continuous, 416 integer (390 binary)
Found heuristic solution: objective 33364.000000

Root relaxation: objective 2.776400e+04, 258 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj 

In [55]:
if model.status == GRB.Status.OPTIMAL:
    # (g) Print the optimal objective function value (minimum cost)
    total_cost = model.ObjVal
    print(f"Optimal Total Cost: {total_cost}")

    # (h) Calculate and print the number of overtime shifts in the optimal schedule
    total_overtime_shifts = sum(o[i, j].X for i in range(NUM_NURSES) for j in range(NUM_SHIFTS))
    print(f"Total Number of Overtime Shifts: {total_overtime_shifts}")
else:
    print("No optimal solution found")

Optimal Total Cost: 27764.0
Total Number of Overtime Shifts: 0.0


In [52]:
if model.status == GRB.Status.OPTIMAL:
    # Print the assignment of nurses to shifts
    print("Shift Assignments:")
    for i in range(NUM_NURSES):
        for j in range(NUM_SHIFTS):
            if x[i, j].X > 0.5:  # Assuming x[i, j] is 1 if nurse i is assigned to shift j
                print(f"Nurse {i+1} is assigned to shift {j+1}")
else:
    print("No optimal solution found")

Shift Assignments:
Nurse 1 is assigned to shift 6
Nurse 1 is assigned to shift 8
Nurse 1 is assigned to shift 10
Nurse 1 is assigned to shift 14
Nurse 2 is assigned to shift 4
Nurse 2 is assigned to shift 8
Nurse 2 is assigned to shift 12
Nurse 3 is assigned to shift 2
Nurse 3 is assigned to shift 4
Nurse 3 is assigned to shift 8
Nurse 4 is assigned to shift 4
Nurse 4 is assigned to shift 7
Nurse 4 is assigned to shift 9
Nurse 4 is assigned to shift 13
Nurse 5 is assigned to shift 6
Nurse 5 is assigned to shift 9
Nurse 5 is assigned to shift 13
Nurse 6 is assigned to shift 3
Nurse 6 is assigned to shift 8
Nurse 6 is assigned to shift 12
Nurse 7 is assigned to shift 2
Nurse 7 is assigned to shift 5
Nurse 7 is assigned to shift 12
Nurse 8 is assigned to shift 3
Nurse 8 is assigned to shift 5
Nurse 8 is assigned to shift 9
Nurse 9 is assigned to shift 2
Nurse 9 is assigned to shift 7
Nurse 9 is assigned to shift 11
Nurse 9 is assigned to shift 13
Nurse 10 is assigned to shift 2
Nurse 10 i

In [53]:
if model.status == GRB.Status.OPTIMAL:
    # Print the total hours and overtime hours for each nurse
    print("\nNurse Work Hours:")
    for i in range(NUM_NURSES):
        total_hours = sum(x[i, j].X * HOURS_PER_SHIFT for j in range(NUM_SHIFTS))
        overtime_hours = sum(o[i, j].X * HOURS_PER_SHIFT for j in range(NUM_SHIFTS))
        print(f"Nurse {i+1}: Total Hours = {total_hours}, Overtime Hours = {overtime_hours}")
else:
    print("No optimal solution found")


Nurse Work Hours:
Nurse 1: Total Hours = 48.0, Overtime Hours = 0.0
Nurse 2: Total Hours = 36.0, Overtime Hours = 0.0
Nurse 3: Total Hours = 36.0, Overtime Hours = 0.0
Nurse 4: Total Hours = 48.0, Overtime Hours = 0.0
Nurse 5: Total Hours = 36.0, Overtime Hours = 0.0
Nurse 6: Total Hours = 36.0, Overtime Hours = 0.0
Nurse 7: Total Hours = 36.0, Overtime Hours = 0.0
Nurse 8: Total Hours = 36.0, Overtime Hours = 0.0
Nurse 9: Total Hours = 48.0, Overtime Hours = 0.0
Nurse 10: Total Hours = 36.0, Overtime Hours = 0.0
Nurse 11: Total Hours = 36.0, Overtime Hours = 0.0
Nurse 12: Total Hours = 36.0, Overtime Hours = 0.0
Nurse 13: Total Hours = 48.0, Overtime Hours = 0.0
Nurse 14: Total Hours = 36.0, Overtime Hours = 0.0
Nurse 15: Total Hours = 36.0, Overtime Hours = 0.0
Nurse 16: Total Hours = 36.0, Overtime Hours = 0.0
Nurse 17: Total Hours = 36.0, Overtime Hours = 0.0
Nurse 18: Total Hours = 36.0, Overtime Hours = 0.0
Nurse 19: Total Hours = 36.0, Overtime Hours = 0.0
Nurse 20: Total Hours

In [54]:
# Print the total cost
total_cost = model.ObjVal
print(f"\nTotal Cost: {total_cost}")


Total Cost: 27764.0
